In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#Transfer Learning Model Analysis

##PreProcessing

In [ ]:
# import libraries
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Dense, MaxPool2D, GlobalAveragePooling2D, Dropout, Flatten
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import numpy as np
import cv2
from tqdm import tqdm
import random
import time
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D,Input,AveragePooling2D
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#listing directories of training and testing images
train_dir="/content/gdrive/MyDrive/RM-Project/Training"
test_dir="/content/gdrive/MyDrive/RM-Project/Testing"

In [ ]:
#preprocessing and generating training validation and testing dataset
from keras.preprocessing.image import ImageDataGenerator
trainDataGen=ImageDataGenerator(
    rescale=1./255, #rescale
    validation_split=0.1, #validation split
    zoom_range=0.2, #zoom to create augmented sample
)
testDataGen=ImageDataGenerator(rescale=1./255) #rescaling

trainGen=trainDataGen.flow_from_directory(train_dir,
                                          target_size=(224,224), #image size
                                          color_mode='rgb', #color mode of image
                                          class_mode='categorical', #label to be categorized
                                          batch_size=128, #specifying batch size
                                          subset='training'
                                         )
testGen=testDataGen.flow_from_directory(test_dir, target_size=(224,224),color_mode='rgb',class_mode='categorical',batch_size=1)

valGen=trainDataGen.flow_from_directory(train_dir, target_size=(224,224),color_mode='rgb',class_mode='categorical',batch_size=128,subset='validation')

##VGG16

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
base_model=VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')

for layer in base_model.layers:
  layer.trainable=False

x=Flatten()(base_model.output)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.4)(x)
x=Dense(1024,activation='relu')(x)
out=Dense(4,activation='softmax')(x)

model=Model(inputs=base_model.inputs,outputs=out)
model.summary()

In [ ]:
callbacks=[EarlyStopping(monitor='val_loss',patience=10)]

model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [ ]:
steps_per_epochs=trainGen.samples//128
print(steps_per_epochs)
validation_steps_=valGen.samples//128
print(validation_steps_)

In [ ]:
start = time.time()
history=model.fit(trainGen,validation_data=valGen,epochs=50,callbacks=callbacks, verbose=1)
end = time.time()
print("Execution time", end-start)

In [ ]:
# Extract training and validation accuracy, and training and validation loss from history object
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Define epochs range based on length of history object
epochs_range = range(len(history.history['accuracy']))

# Create subplots to display accuracy and loss graphs side by side
fig, axs = plt.subplots(1, 2, figsize=(18, 6))

# Plot Training and Validation Accuracy on first subplot
axs[0].plot(epochs_range, acc, label='Training Accuracy', marker='o')
axs[0].plot(epochs_range, val_acc, label='Validation Accuracy', marker='o')
axs[0].set_title('Training and Validation Accuracy')
axs[0].set_xlabel('Epochs')
axs[0].set_ylabel('Accuracy')
axs[0].legend(loc='lower right')

# Plot Training and Validation Loss on second subplot
axs[1].plot(epochs_range, loss, label='Training Loss', marker='o')
axs[1].plot(epochs_range, val_loss, label='Validation Loss', marker='o')
axs[1].set_title('Training and Validation Loss')
axs[1].set_xlabel('Epochs')
axs[1].set_ylabel('Loss')
axs[1].legend(loc='upper right')

# Add vertical lines to indicate best accuracy and lowest loss for training and validation
best_val_acc_epoch = np.argmax(val_acc)
best_val_loss_epoch = np.argmin(val_loss)
best_train_acc_epoch = np.argmax(acc)
best_train_loss_epoch = np.argmin(loss)

axs[0].axvline(x=best_val_acc_epoch, color='gray', linestyle='--')
axs[0].text(best_val_acc_epoch+0.5, val_acc[best_val_acc_epoch]-0.1, 'Best Val Acc', fontsize=10, color='gray')

axs[0].axvline(x=best_train_acc_epoch, color='orange', linestyle='--')
axs[0].text(best_train_acc_epoch+0.5, acc[best_train_acc_epoch]-0.1, 'Best Train Acc', fontsize=10, color='orange')

axs[1].axvline(x=best_val_loss_epoch, color='gray', linestyle='--')
axs[1].text(best_val_loss_epoch+0.5, val_loss[best_val_loss_epoch]+0.1, 'Best Val Loss', fontsize=10, color='gray')

axs[1].axvline(x=best_train_loss_epoch, color='orange', linestyle='--')
axs[1].text(best_train_loss_epoch+0.5, loss[best_train_loss_epoch]+0.1, 'Best Train Loss', fontsize=10, color='orange')

# Add horizontal line to indicate convergence of accuracy and loss
axs[0].axhline(y=1.0, color='red', linestyle='--')
axs[0].text(0, 1.01, 'Converged', fontsize=10, color='red')

axs[1].axhline(y=0.0, color='red', linestyle='--')
axs[1].text(0, -0.1, 'Converged', fontsize=10, color='red')

# Adjust subplots spacing and display the plot
fig.tight_layout(pad=3.0)
plt.show()


In [ ]:
# evaluating the model
model_evaluation = model.evaluate(testGen)

In [ ]:
print(f"Model Accuracy:{model_evaluation[1] *100: 0.2f} %") #Calculating accuracy

##VGG19

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model

# Load pre-trained VGG19 model and set all layers to non-trainable
base_model = VGG19(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(1024, activation='relu')(x)
out = Dense(4, activation='softmax')(x)

# Create the final model by connecting the pre-trained model and custom layers
model = Model(inputs=base_model.inputs, outputs=out)
model.summary()


In [ ]:
callbacks=[EarlyStopping(monitor='val_loss',patience=10)]

model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [ ]:
steps_per_epochs=trainGen.samples//128
print(steps_per_epochs)
validation_steps_=valGen.samples//128
print(validation_steps_)

In [ ]:
start = time.time()
history=model.fit(trainGen,validation_data=valGen,epochs=50,callbacks=callbacks, verbose=1)
end = time.time()
print("Execution time", end-start)

In [ ]:
# Plotting training and validation accuracy and loss

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(history.history['accuracy']))

plt.figure(figsize=(16, 8))

# Plot training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right', fontsize=14)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.title('Training and Validation Accuracy', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

# Plot training and validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right', fontsize=14)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.title('Training and Validation Loss', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.show()


In [ ]:
# evaluating the model
model_evaluation = model.evaluate(testGen)

In [ ]:
print(f"Model Accuracy:{model_evaluation[1] *100: 0.2f} %") #Calculating accuracy

##ResNet50

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
base_model=ResNet50(input_shape=(224,224,3),include_top=False,weights='imagenet')

for layer in base_model.layers:
  layer.trainable=False

x=Flatten()(base_model.output)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.4)(x)
x=Dense(1024,activation='relu')(x)
out=Dense(4,activation='softmax')(x)

model=Model(inputs=base_model.inputs,outputs=out)
model.summary()

In [ ]:
callbacks=[EarlyStopping(monitor='val_loss',patience=10)]

model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [ ]:
steps_per_epochs=trainGen.samples//128
print(steps_per_epochs)
validation_steps_=valGen.samples//128
print(validation_steps_)

In [ ]:
start = time.time()
history=model.fit(trainGen,validation_data=valGen,epochs=50,callbacks=callbacks, verbose=1)
end = time.time()
print("Execution time", end-start)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.history['accuracy']))

plt.figure(figsize=(18, 6))
plt.subplot(1, 2, 1)
plt.plot(range(len(history.history['accuracy'])), acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='upper right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# evaluating the model
model_evaluation = model.evaluate(testGen)

In [ ]:
print(f"Model Accuracy:{model_evaluation[1] *100: 0.2f} %") #Calculating accuracy

##ResNet101

In [ ]:
from tensorflow.keras.applications import ResNet101

In [ ]:
base_model=ResNet101(input_shape=(224,224,3),include_top=False,weights='imagenet')

for layer in base_model.layers:
  layer.trainable=False

x=Flatten()(base_model.output)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.4)(x)
x=Dense(1024,activation='relu')(x)
out=Dense(4,activation='softmax')(x)

model=Model(inputs=base_model.inputs,outputs=out)
model.summary()

In [ ]:
callbacks=[EarlyStopping(monitor='val_loss',patience=10)]

model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [ ]:
steps_per_epochs=trainGen.samples//128
print(steps_per_epochs)
validation_steps_=valGen.samples//128
print(validation_steps_)

In [ ]:
start = time.time()
history=model.fit(trainGen,validation_data=valGen,epochs=50,callbacks=callbacks, verbose=1)
end = time.time()
print("Execution time", end-start)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.history['accuracy']))

plt.figure(figsize=(18, 6))
plt.subplot(1, 2, 1)
plt.plot(range(len(history.history['accuracy'])), acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='upper right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# evaluating the model
model_evaluation = model.evaluate(testGen)

In [ ]:
print(f"Model Accuracy:{model_evaluation[1] *100: 0.2f} %") #Calculating accuracy

##DenseNet121

In [ ]:
from tensorflow.keras.applications import DenseNet121

In [ ]:
base_model=DenseNet121(input_shape=(224,224,3),include_top=False,weights='imagenet')

for layer in base_model.layers:
  layer.trainable=False

x=Flatten()(base_model.output)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.4)(x)
x=Dense(1024,activation='relu')(x)
out=Dense(4,activation='softmax')(x)

model=Model(inputs=base_model.inputs,outputs=out)
model.summary()

In [ ]:
callbacks=[EarlyStopping(monitor='val_loss',patience=10)]

model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [ ]:
steps_per_epochs=trainGen.samples//128
print(steps_per_epochs)
validation_steps_=valGen.samples//128
print(validation_steps_)

In [ ]:
start = time.time()
history=model.fit(trainGen,validation_data=valGen,epochs=50,callbacks=callbacks, verbose=1)
end = time.time()
print("Execution time", end-start)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.history['accuracy']))

plt.figure(figsize=(18, 6))
plt.subplot(1, 2, 1)
plt.plot(range(len(history.history['accuracy'])), acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='upper right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# evaluating the model
model_evaluation = model.evaluate(testGen)

In [ ]:
print(f"Model Accuracy:{model_evaluation[1] *100: 0.2f} %") #Calculating accuracy

##DenseNet201

In [ ]:
from keras.applications.densenet import DenseNet201 #importing libraries

In [ ]:
#creating model
base_model=DenseNet201(input_shape=(224,224,3),include_top=False,weights='imagenet')

for layer in base_model.layers:
  layer.trainable=False

x=Flatten()(base_model.output)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.4)(x)
x=Dense(1024,activation='relu')(x)
out=Dense(4,activation='softmax')(x)

model=Model(inputs=base_model.inputs,outputs=out)
model.summary()

In [ ]:
#adding early stopping as callback and compiling model
callbacks=[EarlyStopping(monitor='val_loss',patience=10)]

model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [ ]:
#creating training and validation steps based on batch size
steps_per_epochs=trainGen.samples//128
print(steps_per_epochs)
validation_steps_=valGen.samples//128
print(validation_steps_)

In [ ]:
#training model and calculating execution time
start = time.time()
history=model.fit(trainGen,validation_data=valGen,epochs=50,callbacks=callbacks, verbose=1)
end = time.time()
print("Execution time", end-start)

In [ ]:
#plotting accuracy and loss graph
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(history.history['accuracy']))

plt.figure(figsize=(18, 6))
plt.subplot(1, 2, 1)
plt.plot(range(len(history.history['accuracy'])), acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='upper right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# evaluatinn the model
model_evaluation = model.evaluate(testGen)

In [ ]:
print(f"Model Accuracy:{model_evaluation[1] *100: 0.2f} %") #Calculating accuracy

# ClassifierAnalysis

##PreProcessing

In [ ]:
train_dir="/content/gdrive/MyDrive/RM-Project/Training"
test_dir="/content/gdrive/MyDrive/RM-Project/Testing"

In [ ]:
import numpy as np
from sklearn.metrics import classification_report,confusion_matrix
import time
import matplotlib.pyplot as plt

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
trainDataGen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1,
    zoom_range=0.2,
    
    
)
testDataGen=ImageDataGenerator(rescale=1./255)

trainGen=trainDataGen.flow_from_directory(train_dir,
                                          target_size=(224,224),
                                          color_mode='rgb',
                                          class_mode='categorical',
                                          batch_size=128,
                                          subset='training'
                                         )
testGen=testDataGen.flow_from_directory(test_dir, target_size=(224,224),color_mode='rgb',class_mode='categorical',batch_size=1)

valGen=trainDataGen.flow_from_directory(train_dir, target_size=(224,224),color_mode='rgb',class_mode='categorical',batch_size=128,subset='validation')

In [ ]:
train_y = []
for i in range(len(trainGen)):
  for j in range(len(trainGen[i])):
    if j == 1:
      for k in trainGen[i][j]:
        train_y.append(k)
print(train_y[0])

In [ ]:
test_y = []
for i in range(len(testGen)):
  for j in range(len(testGen[i])):
    if j == 1:
      for k in testGen[i][j]:
        test_y.append(k)
print(test_y[0])

In [ ]:
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D,Input,AveragePooling2D, MaxPool2D
from keras.applications.densenet import DenseNet201

base_model=DenseNet201(input_shape=(224,224,3),include_top=False,weights='imagenet')

for layer in base_model.layers:
  layer.trainable=False

model = base_model.output
model = MaxPool2D(pool_size=(2,2))(model)
model = Dropout(0.5)(model)
model = Flatten()(model)
model = Model(inputs=base_model.input, outputs=model)
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks=[EarlyStopping(monitor='val_loss',patience=10)]

model.compile(loss="categorical_crossentropy",optimizer="Adam",metrics=["accuracy"])

In [ ]:
x_mod = model.predict(trainGen)

In [ ]:
x_mod.shape

In [ ]:
y_train_up1 = np.argmax(train_y, axis=-1)

In [ ]:
x_testpred = model.predict(testGen)
x_testpred.shape

In [ ]:
y_test_up1 = np.argmax(test_y, axis=-1)

##SVM

In [ ]:
from sklearn import svm 

In [ ]:
import seaborn as sns

In [ ]:
svm_lin = svm.SVC(C=1.0, kernel='linear') #specifying linear kernel for SVM
start = time.time()
svm_lin.fit(x_mod, y_train_up1) #fit the model
end = time.time()
print("Execution time:", end-start)
y_pred = svm_lin.predict(x_testpred) #predicting the model
print(classification_report(y_test_up1, y_pred))

In [ ]:
print(confusion_matrix(y_test_up1,y_pred)) #confusion matrix

In [ ]:
#Heatmap of confusion matrix
fig,ax=plt.subplots(1,1,figsize=(14,7))
sns.heatmap(confusion_matrix(y_test_up1,y_pred),ax=ax,annot=True,
           linewidths=2, cmap='Blues')
fig.text(s='Heatmap of the Confusion Matrix',size=18,fontweight='bold',
             fontname='monospace',y=0.92,x=0.28,)

plt.show()

In [ ]:
#calculating accuracy
count = 0
for i in range(len(y_pred)):
  if y_pred[i] == y_test_up1[i]:
    count +=1
print("Accuracy is :", (count/len(y_pred)))

##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RF = RandomForestClassifier(n_estimators=50, random_state=42) #specifying random forest with 50 trees to be considered for estimation

In [ ]:
start = time.time()
RF.fit(x_mod,y_train_up1)
end = time.time()
print("Execution time:", end-start)

In [32]:
pred_RF = RF.predict(x_testpred)

In [33]:
print(classification_report(y_test_up1, pred_RF))

              precision    recall  f1-score   support

           0       0.69      0.18      0.29       100
           1       0.60      0.75      0.67       115
           2       0.72      0.86      0.78       105
           3       0.63      0.85      0.72        74

    accuracy                           0.65       394
   macro avg       0.66      0.66      0.61       394
weighted avg       0.66      0.65      0.61       394



In [34]:
count = 0
for i in range(len(pred_RF)):
  if pred_RF[i] == y_test_up1[i]:
    count +=1
print("Accuracy is :", (count/len(pred_RF)))

Accuracy is : 0.6522842639593909


##KNN

In [35]:
from sklearn.neighbors import KNeighborsClassifier

In [36]:
neigh = KNeighborsClassifier(n_neighbors=3) #specifying 3 nearest neighbor for knn

In [37]:
start = time.time()
neigh.fit(x_mod,y_train_up1)
end = time.time()
print("Execution time:", end-start)

Execution time: 0.033570289611816406


In [38]:
out_KNN = neigh.predict(x_testpred)

In [39]:
print(classification_report(y_test_up1, out_KNN))

              precision    recall  f1-score   support

           0       0.67      0.32      0.43       100
           1       0.70      0.70      0.70       115
           2       0.75      0.90      0.82       105
           3       0.58      0.84      0.69        74

    accuracy                           0.68       394
   macro avg       0.67      0.69      0.66       394
weighted avg       0.68      0.68      0.66       394



In [40]:
count = 0
for i in range(len(out_KNN)):
  if out_KNN[i] == y_test_up1[i]:
    count +=1
print("Accuracy is :", (count/len(out_KNN)))

Accuracy is : 0.6802030456852792
